### Modeling

#### The purpose of this notebook is to experiment with predictive modeling on pre-processed run data.

In [4]:
import polars as pl

features_path = "../data-processed/run_data_1-16-26_processed.parquet"
features = ["accel_x", "accel_y", "accel_z", "cadence_steps_per_minute", "accel_magnitude_smoothed"]
target = "target_speed_mps_smoothed"
window_size_hz = 50 # 50 rows at 50 Hz = 1 second

df = pl.read_parquet(features_path).select(features + [target])
print(df.shape)
df.head()

(21632, 6)


accel_x,accel_y,accel_z,cadence_steps_per_minute,accel_magnitude_smoothed,target_speed_mps_smoothed
f64,f64,f64,f64,f64,f64
-0.45491,0.162384,-0.308426,180.0,0.499111,0.7956
-0.440277,0.162308,-0.150299,180.0,0.483114,0.816
-0.33432,0.034775,-0.000259,180.0,0.502502,0.8364
-0.428482,-0.061768,-0.045181,180.0,0.581293,0.8568
-0.55397,-0.204468,-0.327637,180.0,0.695047,0.8772


In [5]:
from gait_analytics.model.utils import train_val_split_data

train, val, feature_scaler, target_scaler = train_val_split_data(df, features, target, window_size=window_size_hz)

train_features, train_targets = train
val_features, val_targets = val

print(f"Train features shape: {train_features.shape}")
print(f"Train targets shape: {train_targets.shape}")

print(f"Val features shape: {val_features.shape}")
print(f"Val targets shape: {val_targets.shape}")

Train features shape: (17305, 5)
Train targets shape: (17305, 1)
Val features shape: (4277, 5)
Val targets shape: (4277, 1)


In [6]:
from gait_analytics.model.utils import create_cnn_windows

stride = 1

X_train, y_train = create_cnn_windows(train_features, train_targets, features, window_size=window_size_hz, stride=stride)
X_val, y_val = create_cnn_windows(val_features, val_targets, features, window_size=window_size_hz, stride=stride)

print("X_train shape:", X_train.shape)
print("y_train shape:", y_train.shape)
print("\nX_val shape:", X_val.shape)
print("y_val shape:", y_val.shape)

X_train shape: (17256, 5, 50)
y_train shape: (17256, 1)

X_val shape: (4228, 5, 50)
y_val shape: (4228, 1)


In [9]:
import pytorch_lightning as lit
from pytorch_lightning.loggers import TensorBoardLogger
from torch.utils.data import DataLoader

from gait_analytics.model.gait_lit_module import GaitDataset, GaitLightingModel
from gait_analytics.model.gait_model import GaitNet

learning_rate = 1e-3

train_dataset = GaitDataset(X_train, y_train)
test_dataset = GaitDataset(X_val, y_val)

train_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_dataloader = DataLoader(test_dataset, batch_size=32, shuffle=False)

model = GaitNet(input_channels=len(features))
lit_model = GaitLightingModel(model=model, learning_rate=learning_rate) 

logger = TensorBoardLogger("logs", name="cnn_gait_model")
trainer = lit.Trainer(max_epochs=10, accelerator="auto", logger=logger)

trainer.fit(lit_model, train_dataloader, val_dataloader)

GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
💡 Tip: For seamless cloud logging and experiment tracking, try installing [litlogger](https://pypi.org/project/litlogger/) to enable LitLogger, which logs metrics and artifacts automatically to the Lightning Experiments platform.
💡 Tip: For seamless cloud uploads and versioning, try installing [litmodels](https://pypi.org/project/litmodels/) to enable LitModelCheckpoint, which syncs automatically with the Lightning model registry.

  | Name    | Type    | Params | Mode  | FLOPs
----------------------------------------------------
0 | model   | GaitNet | 44.8 K | train | 0    
1 | loss_fn | MSELoss | 0      | train | 0    
----------------------------------------------------
44.8 K    Trainable params
0         Non-trainable params
44.8 K    Total params
0.179     Total estimated model params size (MB)
17        Modules in train mode
0         Modules in eval mode
0         Total Flops


Epoch 9: 100%|██████████| 540/540 [00:02<00:00, 199.56it/s, v_num=0, train_loss=0.0166, val_loss=0.809]

`Trainer.fit` stopped: `max_epochs=10` reached.


Epoch 9: 100%|██████████| 540/540 [00:02<00:00, 199.01it/s, v_num=0, train_loss=0.0166, val_loss=0.809]
